# 建立神经网络
神经网络由对数据执行操作的层或模块组成。torch.nn命名空间提供了创建自己的神经网络所需的所有构建块。pytorch中的每个模块都对nn.Module进行子类化。神经网络是由其他模块组成的，这种嵌套结构可以轻松构建和管理复杂的体系结构。接下来我们将要建立一个神经网络，对FashionMNIST中的图像进行分类。

In [2]:
import torch
import os
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## 获取设备进行训练
我们希望可以在GPU等硬件加速器上训练我们的模型（如果可用的话）。我们可以先检查torch.cuda是否可用，否则我们将继续使用CPU。

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


## 定义类
我们通过对nn.Module进行子分类来定义神经网络，并在\_\_init\_\_中初始化神经网络层。每一个nn.Module子类都在forward方法中实现对输入数据的操作。

In [26]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [27]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): Softmax(dim=1)
  )
)


为了使用模型，我们将输入数据传入给它，这将执行forward，以及一些后台操作，不要直接调用model.forward()

在输入上调用模型返回一个10维张量，每个类都有原始预测值，我们通过nn.Softmax模块的实例来获得预测概率。

In [11]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
print('pred_probab:', pred_probab)
y_pred = pred_probab.argmax(1)
print('Predicted class:', y_pred)

pred_probab: tensor([[0.0940, 0.1095, 0.0950, 0.0996, 0.1025, 0.0993, 0.1082, 0.0988, 0.0966,
         0.0965]], grad_fn=<SoftmaxBackward>)
Predicted class: tensor([1])


## 模型图层
分解一下刚刚建立的神经网络模型中的层，为了更好地说明这一点，我们将提取一个包含3张尺寸为28x28的图像的样本，看看当我们通过网络时会发生什么

In [13]:
ds = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

ds_loader = DataLoader(ds, batch_size=3, shuffle=True)
imgs, labels = next(iter(ds_loader))
print('shape of imgs:', imgs.size())
print('shape of labels:', labels.size())

shape of imgs: torch.Size([3, 1, 28, 28])
shape of labels: torch.Size([3])


### nn.Flatten()

In [14]:
flatten = nn.Flatten()
flat_img = flatten(imgs)
print(flat_img.size())

torch.Size([3, 784])


### nn.Linear()

In [20]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_img)
print(hidden1.size())

torch.Size([3, 20])


In [21]:
print('Before ReLU:', hidden1)
hidden1 = nn.ReLU()(hidden1)
print('After ReLU:', hidden1)
print('Shape of ')

Before ReLU: tensor([[-0.6577, -0.1392,  0.0400, -0.0337, -0.1029,  0.2172, -0.2490,  0.1131,
          0.2035,  0.2363, -0.2736,  0.3758,  0.1055,  0.1533, -0.1044,  0.0492,
          0.1779, -0.1198,  0.0735, -0.1155],
        [-0.3240,  0.0142, -0.1116, -0.1404,  0.0882,  0.0504, -0.1316, -0.0441,
          0.0477,  0.0129, -0.0542, -0.0412,  0.1936,  0.0915, -0.0917,  0.0899,
          0.1803, -0.0063, -0.1098, -0.1386],
        [-0.4464,  0.3122,  0.1718, -0.0155,  0.5265,  0.0774, -0.4523,  0.1734,
          0.4393,  0.1010, -0.0254,  0.2918,  0.0340,  0.0455, -0.1272, -0.0238,
          0.3351, -0.3508, -0.4378, -0.1336]], grad_fn=<AddmmBackward>)
After ReLU: tensor([[0.0000, 0.0000, 0.0400, 0.0000, 0.0000, 0.2172, 0.0000, 0.1131, 0.2035,
         0.2363, 0.0000, 0.3758, 0.1055, 0.1533, 0.0000, 0.0492, 0.1779, 0.0000,
         0.0735, 0.0000],
        [0.0000, 0.0142, 0.0000, 0.0000, 0.0882, 0.0504, 0.0000, 0.0000, 0.0477,
         0.0129, 0.0000, 0.0000, 0.1936, 0.0915, 0.0000,

In [22]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
logits = seq_modules(imgs)
print(logits)

tensor([[-0.0927,  0.1071,  0.2216,  0.1213, -0.0108,  0.0499,  0.1181,  0.0800,
          0.2860, -0.0242],
        [-0.0870,  0.1946,  0.2125,  0.0502,  0.0557, -0.0037,  0.0763,  0.0561,
          0.2107, -0.0368],
        [-0.1636, -0.0101,  0.3898,  0.2017,  0.0426,  0.0866,  0.1051, -0.0485,
          0.2824,  0.0200]], grad_fn=<AddmmBackward>)


### nn.Softmax()
 神经网络的最后一层线性返回logits，该值的取值范围没有任何限制，这时我们可以使用nn.Softmax模块来将logits的值压缩到【0， 1】的范围内，代表模型对每个类别的预测概率，dim参数表示值相加必须为1的维度

In [24]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print('pred_probab:', pred_probab)

pred_probab: tensor([[0.0832, 0.1016, 0.1139, 0.1030, 0.0903, 0.0959, 0.1027, 0.0989, 0.1215,
         0.0891],
        [0.0848, 0.1124, 0.1144, 0.0973, 0.0978, 0.0922, 0.0999, 0.0979, 0.1142,
         0.0892],
        [0.0766, 0.0893, 0.1332, 0.1104, 0.0942, 0.0984, 0.1002, 0.0860, 0.1197,
         0.0921]], grad_fn=<SoftmaxBackward>)


## 模型参数
神经网络中的许多层都经过了参数化，即具有在训练期间优化的相关权重和偏置。子类nn.Module会自动跟踪模型对象中定义的所有字段，并使用模型的parameter（）或named_parameter（）方法访问所有参数。以刚才建立的模型为例，我们将迭代其中的每个参数，打印出其尺寸和值。

In [28]:
print('The structure of model:', model)

for name, param in model.named_parameters():
    print('Layer:{} Size:{}, Value:{}'.format(name, param.size(), param[:2]))

The structure of model: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): Softmax(dim=1)
  )
)
Layer:linear_relu_stack.0.weight Size:torch.Size([512, 784]), Value:tensor([[-0.0094,  0.0073,  0.0213,  ...,  0.0112,  0.0089,  0.0302],
        [ 0.0165,  0.0310, -0.0004,  ...,  0.0001, -0.0347, -0.0138]],
       grad_fn=<SliceBackward>)
Layer:linear_relu_stack.0.bias Size:torch.Size([512]), Value:tensor([-0.0011, -0.0346], grad_fn=<SliceBackward>)
Layer:linear_relu_stack.2.weight Size:torch.Size([512, 512]), Value:tensor([[ 0.0377, -0.0416, -0.0082,  ...,  0.0164, -0.0125,  0.0241],
        [ 0.0387,  0.0244, -0.0253,  ..., -0.0254,  0.0128,  0.0374]],
       grad_fn=<SliceBackward>)
Layer:linear_relu_stack.2.bias Size:to